In [2]:
%pip install ultralytics



Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from ultralytics import YOLO
import glob

In [4]:
INPUT_DIR = r"C:/Users/user/Documents/AI_project_clone/data_set_helmet/css-data"
WORK_DIR = r"C:/Users/user/Documents/AI_project_clone//working"

In [4]:
import os

# 데이터 경로 설정
LABELS_DIR = os.path.join(INPUT_DIR, 'train/labels')  # train 라벨 디렉토리
ALLOWED_CLASSES = {'0': 'helmet', '2': 'No-helmet', '5': 'person'}  # 유지할 클래스 ID
CLASS_MAPPING = {'0': '0', '2': '1', '5': '2'}  # 새로운 클래스 ID 매핑

def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        
        updated_lines = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id, *coords = line.split()
                if cls_id in ALLOWED_CLASSES:  # 허용된 클래스만 유지
                    new_cls_id = CLASS_MAPPING[cls_id]  # 클래스 ID 재매핑
                    updated_lines.append(f"{new_cls_id} " + " ".join(coords) + "\n")
        
        # 라벨 파일 업데이트
        with open(file_path, 'w') as f:
            f.writelines(updated_lines)

# train, valid, test 디렉토리의 라벨 파일 수정
for mode in ['train', 'valid', 'test']:
    labels_dir = os.path.join(INPUT_DIR, f"{mode}/labels")
    update_labels(labels_dir)

print("라벨 파일 수정이 완료되었습니다.")


라벨 파일 수정이 완료되었습니다.


In [5]:


# WORK_DIR 디렉토리 생성
os.makedirs(WORK_DIR, exist_ok=True)

# 수정된 클래스 설정
num_classes = 3
classes = ['helmet', 'No-helmet', 'Person']  # Hardhat -> helmet, NO-Hardhat -> No-helmet

# data.yaml 파일 생성
dict_file = {
    'train': os.path.join(INPUT_DIR, 'train/images'),
    'val': os.path.join(INPUT_DIR, 'valid/images'),
    'test': os.path.join(INPUT_DIR, 'test/images'),
    'nc': num_classes,
    'names': classes
}

# data.yaml 저장
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'w+') as file:
    yaml.dump(dict_file, file)

print(f"'data.yaml' 파일이 '{data_yaml_path}'에 저장되었습니다.")

# 라벨 데이터 수정 (Hardhat -> helmet, NO-Hardhat -> No-helmet, Person 그대로, 나머지 제거)
ALLOWED_CLASSES = {'0': 'helmet', '2': 'No-helmet', '5': 'Person'}  # 기존 클래스 ID 매핑
CLASS_MAPPING = {'0': '0', '2': '1', '5': '2'}  # 새로운 클래스 ID 매핑


def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        
        updated_lines = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id, *coords = line.split()
                if cls_id in ALLOWED_CLASSES:  # 허용된 클래스만 유지
                    new_cls_id = CLASS_MAPPING[cls_id]  # 클래스 ID 재매핑
                    updated_lines.append(f"{new_cls_id} " + " ".join(coords) + "\n")
        
        # 라벨 파일 업데이트
        with open(file_path, 'w') as f:
            f.writelines(updated_lines)


# train, valid, test 디렉토리의 라벨 파일 수정
for mode in ['train', 'valid', 'test']:
    labels_dir = os.path.join(INPUT_DIR, f"{mode}/labels")
    update_labels(labels_dir)

print("라벨 파일 수정이 완료되었습니다.")

# 데이터셋 크기 출력
for mode in ['train', 'valid', 'test']:
    files = glob.glob(os.path.join(INPUT_DIR, mode, 'images', '*'))
    print(f'{mode} set size: {len(files)}\n')

# YOLOv8 모델 학습
model = YOLO('yolov8n.pt')

model.train(
    data=data_yaml_path,
    task='detect',
    imgsz=640,
    epochs=30,
    batch=16,
    mode='train',
    name='yolov8n_v1_train'
)

print("Training completed!")


'data.yaml' 파일이 'C:/Users/user/Documents/AI_project_clone//working\data.yaml'에 저장되었습니다.
라벨 파일 수정이 완료되었습니다.
train set size: 2605

valid set size: 114

test set size: 82

New https://pypi.org/project/ultralytics/8.3.63 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/user/Documents/AI_project_clone//working\data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v1_train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7

train: Scanning C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\train\labels... 2605 images, 1291 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:06<00:00, 404.28it/s]


train: New cache created: C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\train\labels.cache


val: Scanning C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\valid\labels... 114 images, 72 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<00:00, 287.30it/s]

val: New cache created: C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\valid\labels.cache


...

      30/30      2.32G      1.126     0.7436       1.04         22        640: 100%|██████████| 163/163 [00:58<00:00,  2.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.02it/s]

                   all        114         79      0.949      0.747      0.855      0.508



30 epochs completed in 0.506 hours.
Optimizer stripped from runs\detect\yolov8n_v1_train5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolov8n_v1_train5\weights\best.pt, 6.2MB

Validating runs\detect\yolov8n_v1_train5\weights\best.pt...
Ultralytics 8.3.62  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.01it/s]


                   all        114         79      0.949      0.747      0.854      0.507
                helmet         42         79      0.949      0.747      0.854      0.507
Speed: 0.5ms preprocess, 6.8ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\yolov8n_v1_train5
Training completed!


In [13]:
from ultralytics import YOLO
import cv2

# 학습된 모델 로드
model = YOLO('runs/detect/yolov8n_v1_train/weights/best.pt')  # 학습 완료된 모델 경로

# OpenCV를 사용하여 웹캠 열기
cap = cv2.VideoCapture(0)  # 0은 기본 카메라. 다른 카메라를 사용하려면 1 또는 다른 번호로 변경

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 카메라에서 실시간 객체 탐지
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # YOLO 모델로 프레임 추론
    results = model.predict(source=frame, save=False, show=False)

    # 탐지 결과를 그리기
    annotated_frame = results[0].plot()  # 결과를 그린 프레임

    # 화면에 표시
    cv2.imshow('YOLOv8 Real-time Detection', annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 helmet, 1 No-helmet, 18.2ms
Speed: 0.0ms preprocess, 18.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 1 No-helmet, 17.3ms
Speed: 1.0ms preprocess, 17.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 3 No-helmets, 18.2ms
Speed: 4.5ms preprocess, 18.2ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 13.4ms
Speed: 3.0ms preprocess, 13.4ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 10.4ms
Speed: 3.0ms preprocess, 10.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 10.4ms
Speed: 5.1ms preprocess, 10.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 10.2ms
Speed: 1.8ms preprocess, 10.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 

In [21]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


## with helmet,no helmet

In [23]:
from ultralytics import YOLO

# 데이터 경로 및 설정
data_yaml_path = r"C:\Users\user\Documents\AI_project_clone\dataset\data.yaml"  # YAML 파일 경로 (수정 필요)
model_checkpoint = 'yolov8n.pt'  # YOLOv8 Nano 모델 가중치 (기본값)

# YOLO 모델 초기화
model = YOLO(model_checkpoint)

# 모델 학습
model.train(
    data=data_yaml_path,
    epochs=30,
    imgsz=640,
    batch=16,
    lr0=0.01,  # 초기 학습률 설정
    name='helmet_detection_model',
    project='helmet_detection'
)


print("학습완료")


New https://pypi.org/project/ultralytics/8.3.63 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\user\Documents\AI_project_clone\dataset\data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=helmet_detection, name=helmet_detection_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cla

train: Scanning C:\Users\user\Documents\AI_project_clone\dataset\train\labels.cache... 1098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1098/1098 [00:00<?, ?it/s]
val: Scanning C:\Users\user\Documents\AI_project_clone\dataset\valid\labels.cache... 312 images, 0 backgrounds, 0 corrupt: 100%|██████████| 312/312 [00:00<?, ?it/s]


Plotting labels to helmet_detection\helmet_detection_model\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 6 dataloader workers
Logging results to helmet_detection\helmet_detection_model
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      2.38G     0.8726      1.931      1.181         25        640: 100%|██████████| 69/69 [00:25<00:00,  2.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.29it/s]

                   all        312        312      0.934      0.728      0.921      0.696


...

      30/30      2.34G     0.3715     0.2316     0.8919         10        640: 100%|██████████| 69/69 [00:25<00:00,  2.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.67it/s]

                   all        312        312      0.996      0.981      0.993      0.912



30 epochs completed in 0.243 hours.
Optimizer stripped from helmet_detection\helmet_detection_model\weights\last.pt, 6.2MB
Optimizer stripped from helmet_detection\helmet_detection_model\weights\best.pt, 6.2MB

Validating helmet_detection\helmet_detection_model\weights\best.pt...
Ultralytics 8.3.62  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.56it/s]


                   all        312        312      0.991      0.982      0.994      0.914
             no helmet        152        152      0.983          1      0.995      0.907
           with helmet        160        160          1      0.963      0.993      0.921
Speed: 0.6ms preprocess, 4.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to helmet_detection\helmet_detection_model
학습완료


## 로그인창

In [44]:
%pip install mediapipe
%pip install pyautogui

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements 

In [46]:
import cv2
import time
import mediapipe as mp
import pyautogui
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 화면 크기 확인
screen_w, screen_h = pyautogui.size()

# 헬멧 감지 및 로그인 상태 변수 초기화
helmet_detected_time = None
eye_blink_detected = False
login_shown = False

# 로그인 창 표시 함수
def show_login_window():
    global login_shown
    if not login_shown:
        from tkinter import Tk, simpledialog
        root = Tk()
        root.withdraw()
        simpledialog.askstring("Login", "Please log in to continue:")
        login_shown = True
        root.destroy()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_blink_detected = True  # 눈 깜빡임 감지
        else:
            eye_blink_detected = False

    # YOLO로 헬멧 탐지
    results = model.predict(source=frame, save=False, show=False)

    helmet_detected = False
    for box in results[0].boxes:
        cls = int(box.cls)
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])

        # 바운딩 박스 그리기
        if cls == 0:  # No-helmet
            color = (0, 0, 255)  # 빨간색
            label = "No helmet"
            helmet_detected_time = None
        elif cls == 1:  # With-helmet
            color = (0, 255, 0)  # 초록색
            label = "With helmet"
            helmet_detected = True  # 헬멧 감지
            if helmet_detected_time is None:
                helmet_detected_time = time.time()

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            frame,
            f"{label} ({conf:.2f})",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
        )

    # 눈 깜빡임과 헬멧 착용 감지가 모두 충족되었는지 확인
    if eye_blink_detected and helmet_detected and helmet_detected_time:
        if time.time() - helmet_detected_time >= 3:  # 3초 이상 헬멧 감지
            show_login_window()  # 로그인 창 표시

    # 화면 상단 메시지 표시
    if not helmet_detected:
        cv2.putText(frame, "Please wear a helmet", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
    elif helmet_detected:
        cv2.putText(frame, "Helmet detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet and Eye Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 no helmet, 24.7ms
Speed: 0.0ms preprocess, 24.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 9.8ms
Speed: 4.3ms preprocess, 9.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 2.1ms
Speed: 1.0ms preprocess, 2.1ms inference, 8.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 11.9ms
Speed: 5.9ms preprocess, 11.9ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 2.1ms
Speed: 4.0ms preprocess, 2.1ms inference, 6.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 7.3ms
Speed: 2.0ms preprocess, 7.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 no helmet, 8.2ms
Speed: 1.5ms preprocess, 8.2ms inference, 0.0ms postprocess per image at s

## 착용시간이 3초 지속되면 qr 코드가 켜지도록

In [ ]:
import cv2
import time
from ultralytics import YOLO

# QR 코드 이미지 경로
qr_image_path = "qr_code.png"  # 생성된 QR 코드 이미지 파일 경로

# 학습된 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# OpenCV를 사용하여 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# QR 코드 이미지 로드
qr_image = cv2.imread(qr_image_path)
if qr_image is None:
    print(f"Error: Could not load QR code image from {qr_image_path}")
    exit()

# QR 코드 표시 여부 플래그와 헬멧 상태 타이머 초기화
show_qr_code = False
helmet_detected_time = None  # 헬멧이 처음 감지된 시간을 저장

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # YOLO 모델로 프레임 추론
    results = model.predict(source=frame, save=False, show=False)

    # 헬멧 착용 여부 확인 변수
    helmet_detected = False
    no_helmet_detected = False

    # 탐지 결과 처리
    for box in results[0].boxes:
        cls = int(box.cls)  # 클래스 ID
        conf = float(box.conf[0])  # 신뢰도
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표

        if cls == 0:  # No-helmet
            label = "No helmet"
            no_helmet_detected = True  # 헬멧 미착용 상태 감지
            color = (0, 0, 255)  # 빨간색
            helmet_detected_time = None  # 헬멧 감지 타이머 초기화
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        elif cls == 1:  # With-helmet
            label = "With helmet"
            helmet_detected = True  # 헬멧 착용 상태 감지
            color = (0, 255, 0)  # 초록색
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # 라벨 텍스트 표시
        cv2.putText(
            frame,
            f"{label} ({conf:.2f})",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
        )

    # 헬멧 감지 타이머 로직
    if helmet_detected:
        if helmet_detected_time is None:
            helmet_detected_time = time.time()  # 처음 헬멧이 감지된 시간 기록
        elif time.time() - helmet_detected_time >= 3:  # 3초 연속 감지 확인
            # QR 코드 표시
            if not show_qr_code:
                print("Displaying QR Code. Scan to visit the website.")
                cv2.imshow("QR Code", qr_image)
                show_qr_code = True

    else:
        helmet_detected_time = None  # 헬멧 감지 리셋
        if show_qr_code:
            cv2.destroyWindow("QR Code")
            show_qr_code = False

    # 화면 상단에 메시지 표시
    if no_helmet_detected:
        cv2.putText(frame, "Please wear a helmet", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    elif helmet_detected:
        cv2.putText(frame, "Helmet detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
